In [ ]:
# import sys
# !{sys.executable} -m pip install package_name

## import packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.impute import KNNImputer
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from sklearn.preprocessing import MinMaxScaler
import warnings

from pygam import LinearGAM, s, te
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

## load dataset

In [ ]:
original_dat = pd.read_csv('single_housing.csv')

In [ ]:
original_dat

In [ ]:
#create duplicate dataset
dat = original_dat

## data cleaning

In [ ]:
sns.boxplot(data=dat, x='Price')

In [ ]:
#remove outliers
percentile25 = dat['Price'].quantile(0.25)
percentile75 = dat['Price'].quantile(0.75)
IQR = percentile75 - percentile25
dat = dat.loc[(dat['Price']>percentile25-1.5*IQR) & (dat['Price']<percentile75+1.5*IQR),]

In [ ]:
#number of rows removed
len(original_dat) - len(dat)

In [ ]:
#remove non-numeric columns
dat = dat.drop(['Street','City','State','Zip','geoadd','CheckAddDuplicate'], axis=1)

In [ ]:
dat.describe()

In [ ]:
dat.isna().sum()

In [ ]:
sns.histplot(data=dat, x='Price', bins=10)

In [ ]:
#dataset explainations
#original_dat (uncleaned dataset)
#dat (cleaned dataset with NA values)
#the response variable does not have NAs
y = dat['Price'].values
X = dat.iloc[: , 1:]
#X_simple (filled NAs with column means)
#X_knn (filled NAs with KNNImputer)
#X_forest (folled NAs with MissForest)

## simple approach (fill NAs)

In [ ]:
X_simple = X.fillna(X.mean()).values
gam_reg = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5)).fit(X_simple, y)
cv_score = sqrt(gam_reg.statistics_['GCV'])
cv_score

## KNNImputer (fill NAs)

In [ ]:
#a different approach to NA
k = list(range(5,100,5)) + list(range(100,501,50))
outsample = []

for i in range(len(k)):
    #create duplicate predictor dataset with missing values
    X_temp = X
    #build the model
    imputer = KNNImputer(n_neighbors=k[i])
    X_temp = pd.DataFrame(imputer.fit_transform(X_temp)).values
    #evaluate on gam model
    gam_reg = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5)).fit(X_temp, y)
    cv_score = sqrt(gam_reg.statistics_['GCV'])
    outsample.append(cv_score)

In [ ]:
knn_table = pd.DataFrame({'k':k, 'out-of-sample error':outsample})
sns.lineplot(data=knn_table, x='k', y='out-of-sample error')

In [ ]:
#select optimal k to finalize the model
imputer = KNNImputer(n_neighbors=100)
X_knn = pd.DataFrame(imputer.fit_transform(X), columns=list(X.columns))

## MissForest (fill NAs)

In [ ]:
imputer = MissForest()
X_forest = pd.DataFrame(imputer.fit_transform(X), columns=list(X.columns))

In [ ]:
gam_reg = LinearGAM(s(0) + s(1) + s(2) + s(3) + s(4) + s(5)).fit(X_forest, y)
cv_score = sqrt(gam_reg.statistics_['GCV'])
cv_score

## KNN (not scaled)

In [ ]:
#prep the data
X = X_forest[['SqFt','Acreage','Beds','Baths']].values
# X = X_knn[['Latitude','Longitude']].values
# X = X_knn.values

In [ ]:
#tune the model
outsample = []
k = list(range(1,20)) + list(range(20,100,5)) + list(range(100,501,100))

for i in range(len(k)):
    knn_model = KNeighborsRegressor(n_neighbors=k[i])
    cv_score = cross_validate(knn_model, X, y, cv=10, scoring='neg_mean_squared_error')['test_score']
    outsample.append(np.mean(np.sqrt(-cv_score)))

In [ ]:
knn_table = pd.DataFrame({'k':k, 'out-of-sample error':outsample})
sns.lineplot(data=knn_table, x='k', y='out-of-sample error')

In [ ]:
optimalk = k[outsample.index(min(outsample))]
print('The best k for out-of-sample prediction: ' + str(optimalk))
print('The best cv out-of-sample error: ' + str(round(min(outsample),3)))

## KNN (scaled)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_forest)
X = scaler.transform(X_forest)

In [ ]:
pd.DataFrame(X, columns=list(X_forest.columns)).describe()

In [ ]:
#tune the model
outsample = []
k = list(range(1,50)) + list(range(50,501,50))

for i in range(len(k)):
    knn_model = KNeighborsRegressor(n_neighbors=k[i])
    cv_score = cross_validate(knn_model, X, y, cv=10, scoring='neg_mean_squared_error')['test_score']
    outsample.append(np.mean(np.sqrt(-cv_score)))

In [ ]:
knn_table = pd.DataFrame({'k':k, 'out-of-sample error':outsample})
sns.lineplot(data=knn_table, x='k', y='out-of-sample error')

In [ ]:
optimalk = k[outsample.index(min(outsample))]
print('The best k for out-of-sample prediction: ' + str(optimalk))
print('The best cv out-of-sample error: ' + str(round(min(outsample),3)))

## split the dataset (too much computation cost to do CV)

In [ ]:
#split the data
X = X_forest.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12345)

## random forest

In [ ]:
#tune the model
maxfeatures = [2,3,4,5,6]
samplesleaf = list(range(1,15)) + list(range(15,50,5))
bestmaxfeature = 99999
bestsamplesleaf = 99999
best_outsample = 99999

for i in tqdm(range(len(maxfeatures))):
    for j in range(len(samplesleaf)):
        rf_model = RandomForestRegressor(n_estimators=500, 
                                         max_features=maxfeatures[i], 
                                         min_samples_leaf=samplesleaf[j])
        rf_model.fit(X_train, y_train)
        #out-of-sample
        test_preds = rf_model.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, test_preds))
        if rmse < best_outsample:
            bestmaxfeature = maxfeatures[i]
            bestsamplesleaf = samplesleaf[j]
            best_outsample = rmse

In [ ]:
print('The best max_features for out-of-sample prediction: ' + str(bestmaxfeature))
print('The best min_samples_leaf for out-of-sample prediction: ' + str(bestsamplesleaf))
print('The best out-of-sample error: ' + str(round(best_outsample,3)))

In [ ]:
#computation time: 5 min.
#best max_features: 6
#best min_samples_leaf: 3
#best out-of-sample error: 267.38

## interaction b/t longitude & latitude

In [ ]:
xlist = list(np.linspace(min(X_forest['Latitude']), max(X_forest['Latitude']), 50))
ylist = list(np.linspace(min(X_forest['Longitude']), max(X_forest['Longitude']), 50))

In [ ]:
temp_table = pd.DataFrame({'Latitude':xlist*50, 'Longitude':np.repeat(ylist,50)})
temp_table['SqFt'] = np.mean(X_forest['SqFt'])
temp_table['Acreage'] = np.mean(X_forest['Acreage'])
temp_table['Beds'] = np.mean(X_forest['Beds'])
temp_table['Baths'] = np.mean(X_forest['Baths'])
temp_table = temp_table.reindex(columns = list(X_forest.columns))
ada_model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=10), 
                              n_estimators=500, 
                              learning_rate=0.05)
ada_model.fit(X_train, y_train)
temp_table['Price'] = ada_model.predict(temp_table)
temp_table

In [ ]:
fig = go.Figure(data =
    go.Contour(
        z=temp_table['Price'],
        x=temp_table['Latitude'], # horizontal axis
        y=temp_table['Longitude'] # vertical axis
    ))
fig.show()

## adaboost

In [ ]:
#tune the model
base_model = list(range(1,16))
tree_num = [500,1000]
learning_rate = [0.001,0.005,0.01,0.05,0.1,0.5]
best_outsample = 99999
best_base_model = 99999
best_tree_num = 99999
best_learning_rate = 99999

for i in tqdm(range(len(base_model))):
    for j in range(len(tree_num)):
        for k in range(len(learning_rate)):
            ada_model = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=base_model[i]), 
                                          n_estimators=tree_num[j], 
                                          learning_rate=learning_rate[k])
            ada_model.fit(X_train, y_train)
            #out-of-sample
            test_preds = ada_model.predict(X_test)
            rmse = sqrt(mean_squared_error(y_test, test_preds))
            if rmse < best_outsample:
                best_outsample = rmse
                best_base_model = base_model[i]
                best_tree_num = tree_num[j]
                best_learning_rate = learning_rate[k]

In [ ]:
print('The best base_model_depth for out-of-sample prediction: ' + str(best_base_model))
print('The best tree_num for out-of-sample prediction: ' + str(best_tree_num))
print('The best learning_rate for out-of-sample prediction: ' + str(best_learning_rate))
print('The best out-of-sample error: ' + str(round(best_outsample,3)))

In [ ]:
#computation time: 24 min.
#best base_model_depth: 10
#best tree_num: 500
#best learning_rate: 0.05
#best out-of-sample error: 266.38

## gradient boost

In [ ]:
#tune the model
maxdepth = [1,2,3,4,5,6,7,8,9,10]
tree_num = [500,1000,1500]
learning_rate = [0.001,0.005,0.01,0.05,0.1,0.5]
best_outsample = 99999
best_maxdepth = 99999
best_tree_num = 99999
best_learning_rate = 99999

for i in tqdm(range(len(maxdepth))):
    for j in range(len(tree_num)):
        for k in range(len(learning_rate)):
            grad_model = GradientBoostingRegressor(max_depth=maxdepth[i], 
                                                   n_estimators=tree_num[j], 
                                                   learning_rate=learning_rate[k])
            grad_model.fit(X_train, y_train)
            #out-of-sample
            test_preds = grad_model.predict(X_test)
            rmse = sqrt(mean_squared_error(y_test, test_preds))
            if rmse < best_outsample:
                best_outsample = rmse
                best_maxdepth = maxdepth[i]
                best_tree_num = tree_num[j]
                best_learning_rate = learning_rate[k]

In [ ]:
print('The best max_depth for out-of-sample prediction: ' + str(best_maxdepth))
print('The best tree_num for out-of-sample prediction: ' + str(best_tree_num))
print('The best learning_rate for out-of-sample prediction: ' + str(best_learning_rate))
print('The best out-of-sample error: ' + str(round(best_outsample,3)))

In [ ]:
#computation time: 21 min.
#best max_depth: 6
#best tree_num: 1500
#best learning_rate: 0.005
#best out-of-sample error: 265.98

## MLP neural network

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
pd.DataFrame(X_train_scaled, columns=list(X_forest.columns)).describe()

In [ ]:
pd.DataFrame(X_test_scaled, columns=list(X_forest.columns)).describe()

In [ ]:
warnings.simplefilter('ignore')
#tune the model
hiddenlayer = [(10,),(20,),(30,),(40,),(50,),(60,),(70,),(80,),(90,),(100,),
               (10,10,),(20,20,),(30,30,),(40,40,),(50,50,),]
activation = ['identity', 'logistic', 'tanh', 'relu']
best_outsample = 99999
best_hiddenlayer = 99999
best_activation = 99999

for i in tqdm(range(len(hiddenlayer))):
    for j in range(len(activation)):
        nn_model = MLPRegressor(hidden_layer_sizes=hiddenlayer[i], 
                                activation=activation[j], 
                                learning_rate_init=0.005, 
                                max_iter=10000, 
                                random_state=12345)
        nn_model.fit(X_train_scaled, y_train)
        #out-of-sample
        test_preds = nn_model.predict(X_test_scaled)
        rmse = sqrt(mean_squared_error(y_test, test_preds))
        if rmse < best_outsample:
            best_outsample = rmse
            best_activation = activation[j]
            best_hiddenlayer = hiddenlayer[i]

In [ ]:
print('The best hiddenlayer for out-of-sample prediction: ' + str(best_hiddenlayer))
print('The best activation for out-of-sample prediction: ' + str(best_activation))
print('The best out-of-sample error: ' + str(round(best_outsample,3)))

In [ ]:
#computation time: 32 min.
#best hiddenlayer: (90,)
#best activation: 'tanh'
#best out-of-sample error: 271.31

In [ ]:
#thoughts
#1. KNNImputer and MissForest to fill NAs (already done)
#2. KNN and rf to predict prices (already done)
#3. learn boosting tree (adaboost and gradientboost) and neural network (MLP) to predict prices (already done)
#4. visualize prediction prices on (longitude, latitude) holding the other variables constant (already done)

#aside
#the dataset is raw, we need to remove the outliers (already done)
#need to add CV measures on some models (already done)
#the ultimate goal is to try to make better predictions on price than we did in summer

#final deliverable (a short paper)
#introduction
#table of model performance
#table of model computation time
#plot of best models performance
#explaination of best models